In [1]:
import json
import sys
import os

from easydict import EasyDict as edict
import pymysql as sql

## 하위 폴더에 있는 패키지 로딩을 위한 밑작업
SEP       = os.path.sep
MISC_PATH = SEP.join(os.getcwd().split(SEP)[:-2])
ROOT_PATH = SEP.join(os.getcwd().split(SEP)[:-4])
sys.path.append(MISC_PATH)

from misc.config import *
from misc.utils import *

[INFO] config.json 파일을 로딩합니다.
[INFO] ports.json 파일을 로딩합니다.


In [2]:
host   = CONFIGS.global_host
port   = PORTS.sql_port
user   = CONFIGS.sql_user
passwd = CONFIGS.sql_passwd

## 노트북 재시작할때마다 DB삭제용 플래그
not_first = True

In [3]:
## DB 서버에 연결
conn   = sql.connect(host     = host, user = user, port = port,
                     password = passwd, db = 'study')

cursor = conn.cursor()

## **1. 집계 함수**
- 함수는 입력값으로 부터 모종의 계산을 하여 출력값을 리턴하는 장치이다.
- 집계 함수는 복수개의 레코드에 대해 집합적인 계산을 수행하여 평균같은 통계값을 산출한다.
### 1-1. COUNT
- COUNT함수는 레코드의 개수를 반환해주는 함수이다.
  ~~~SQL
      SELECT COUNT([필드명]) FROM [테이블명];
  ~~~
  
- WHERE 절과 함께 사용하면 조건에 맞는 데이터의 수를 계산할 수 있다.
  ~~~SQL
      SELECT COUNT([필드명]) FROM [테이블명] WHERE [조건];
  ~~~

In [4]:
## 아래 쿼리문은 총 직원수를 출력해준다.
select(cursor, text = 'count 함수 테스트', 
       column = 'count(*)', table_name = 'tStaff')

## 아래 쿼리문은 월급이 400 이상인 직원 수를 반환해준다.
select(cursor, text = '월급 400이상인 직원의 수', column = 'count(*)', 
       table_name = 'tStaff', cond = 'salary >= 400')

select(cursor, text = '위 쿼리문이 제대로 들어갔는지 확인',
      column = 'name, salary', table_name = 'tStaff', 
      cond   = 'salary >= 400')

[query] select count(*) from tStaff;

[count 함수 테스트]
(20,)


[query] select count(*) from tStaff where salary >= 400;

[월급 400이상인 직원의 수]
(3,)


[query] select name, salary from tStaff where salary >= 400;

[위 쿼리문이 제대로 들어갔는지 확인]
('김유신', 420)
('신사임당', 400)
('장보고', 440)




- 중복된 데이터를 제거한 데이터의 개수를 구하고 싶으면, 필드명 앞에 distinct를 추가한다.
~~~SQL
   select COUNT(distinct [필드명]) from [테이블명];
~~~
- count 함수는 필드값이 NULL인 레코드는 집계에서 제거한다.

In [5]:
## 아래 쿼리문은 중복 데이터를 제거한 데이터의 개수만 집계한다.
select(cursor, text = '중복 데이터 제거', table_name = 'tStaff',
      column = 'count(distinct depart)')

select(cursor, text = '중복 데이터 제거 확인', table_name = 'tStaff', 
       column = 'distinct depart')

## 아래 쿼리문은 NULL 데이터를 제외한 데이터의 개수만 집계한다.
select(cursor, text = 'NULL 값 집계 안함', table_name = 'tStaff',
       column = 'count(score)')

## 총 20개 데이터에서 NULL 데이터 2개 뺀 18개
select(cursor, text = 'NULL 제외한 데이터 개수 확인', 
       table_name = 'tStaff', column = 'score')


## 아래 두 쿼리문은 score가 NULL인 녀석들 집계해준다.
select(cursor, text = 'score가 NULL인 녀석들 수를 집계 1',
       column = 'count(*)', table_name = 'tStaff',
       cond = 'score is null')

select(cursor, text = 'score가 NULL인 녀석들 수를 집계 2',
       column = 'count(*) - count(score)', table_name = 'tStaff')

[query] select count(distinct depart) from tStaff;

[중복 데이터 제거]
(3,)


[query] select distinct depart from tStaff;

[중복 데이터 제거 확인]
('영업부',)
('총무부',)
('인사과',)


[query] select count(score) from tStaff;

[NULL 값 집계 안함]
(18,)


[query] select score from tStaff;

[NULL 제외한 데이터 개수 확인]
(Decimal('56.00'),)
(Decimal('88.80'),)
(Decimal('46.20'),)
(Decimal('49.90'),)
(Decimal('45.10'),)
(Decimal('87.75'),)
(Decimal('92.00'),)
(Decimal('76.50'),)
(Decimal('74.20'),)
(None,)
(Decimal('71.25'),)
(None,)
(Decimal('50.00'),)
(Decimal('65.40'),)
(Decimal('58.30'),)
(Decimal('89.50'),)
(Decimal('69.80'),)
(Decimal('44.50'),)
(Decimal('77.70'),)
(Decimal('52.50'),)


[query] select count(*) from tStaff where score is null;

[score가 NULL인 녀석들 수를 집계 1]
(2,)


[query] select count(*) - count(score) from tStaff;

[score가 NULL인 녀석들 수를 집계 2]
(2,)




##### **연습 문제 #001.** 

> Q1. 실적도 없이 놀고 있는 두 직원은 누구인지 목록을 출력하는 쿼리를 작성하라.  
> Q2. 성취도가 80점 이상인 직원이 몇 명이나 되는지 조사하라.

In [6]:
select(cursor, text='A1.', table_name = 'tStaff',
       cond = 'score is null', column = 'name, depart, score, salary')

select(cursor, text='A2.', table_name = 'tStaff',
       cond = 'score >= 80', column = 'count(*)')

select(cursor, text='A2. REF', table_name = 'tStaff',
       cond = 'score >= 80', column = 'name, depart, score, salary')

[query] select name, depart, score, salary from tStaff where score is null;

[A1.]
('유관순', '영업부', None, 380)
('을지문덕', '영업부', None, 330)


[query] select count(*) from tStaff where score >= 80;

[A2.]
(4,)


[query] select name, depart, score, salary from tStaff where score >= 80;

[A2. REF]
('김유신', '총무부', Decimal('88.80'), 420)
('성삼문', '영업부', Decimal('87.75'), 285)
('신사임당', '영업부', Decimal('92.00'), 400)
('정몽주', '총무부', Decimal('89.50'), 370)




### 1-2. 통계값
||함수|설명|
|---|---|---|
|1|SUM|총합을 구하는 함수|
|2|AVG|평균을 구하는 함수|
|3|MAX|최대값을 구하는 함수|
|4|MIN|최소값을 구하는 함수|
|5|STDDEV|표준 편자를 구하는 함수|
|6|VARIANCE|분산을 구하는 함수|

- SQL Server에서 표준편차와 분산을 구하는 함수는 각각 STDEV, VAR이다.
- COUNT 함수와 마찬가지로 뒤에 WHERE 절을 붙이면 조건에 해당하는 데이터만 집계해준다.

In [7]:
## 아래 쿼리문은 테이블에 있는 도시들의 인구 총합과 평균을 구해준다.
select(cursor, text = '인구 총합 및 평균', table_name = 'tCity',
      column = 'sum(popu), avg(popu)')


## 아래 쿼리문은 면적의 최대값과 최소값을 구해준다.
select(cursor, text = '면적의 최소, 최대', table_name = 'tCity',
       column = 'min(area), max(area)')


## 아래 쿼리문은 인사과의 성취도 합계, 평균을 구해준다.
select(cursor, text = '인사과 성취도의 합계, 평균', table_name = 'tStaff',
       column = 'sum(score), avg(score)', cond = 'depart = "인사과"')

## 아래 쿼리문은 영업부의 최소, 최대 월급을 구해준다.
select(cursor, text = '영업부 월급 최소, 최대', table_name = 'tStaff',
       column = 'min(salary), max(salary)', cond = 'depart = "영업부"')

[query] select sum(popu), avg(popu) from tCity;

[인구 총합 및 평균]
(Decimal('1546'), Decimal('193.2500'))


[query] select min(area), max(area) from tCity;

[면적의 최소, 최대]
(42, 1819)


[query] select sum(score), avg(score) from tStaff where depart = "인사과";

[인사과 성취도의 합계, 평균]
(Decimal('400.80'), Decimal('57.257143'))


[query] select min(salary), max(salary) from tStaff where depart = "영업부";

[영업부 월급 최소, 최대]
(285, 400)




- 문자열이나 날짜 데이터는 MIN, MAX 함수만 사용할 수 있다.
- 아래 코드는 정상적으로 처리될 것 같지만 max(popu)는 출력 필드는 1개,
  name 출력 필드는 여러개로 나오기 때문에 처리되지 않는다.
    ~~~SQL
        SELECT MAX(popu), name from tCity;
    ~~~

In [8]:
## 아래 쿼리문은 에러를 발생시킨다.
try:
    select(cursor, text = '텍스트 합계 에러', table_name = 'tStaff',
          column = 'sum(name)')
except Exception as e: print(f'[에러] {e}')

## 아래 쿼리문은 직원 이름 중 가장 빠른 직원을 반환한다.
select(cursor, text = '직원이름 중 가장 빠른 직원', table_name = 'tStaff',
       column = 'min(name)')

## 아래 쿼리문은 위 쿼리문과 동일하게 동작하지만, 쿼리가 너무 길다.
select(cursor, text = '직원이름 중 가장 빠른 직원 2', table_name = 'tStaff',
       column = 'name', order = 'name asc', limit_k = 1)

[query] select sum(name) from tStaff;

[텍스트 합계 에러]
(0.0,)


[query] select min(name) from tStaff;

[직원이름 중 가장 빠른 직원]
('강감찬',)


[query] select name from tStaff order by name asc limit 1;

[직원이름 중 가장 빠른 직원 2]
('강감찬',)




##### **연습 문제 #002.** 

> Q1. 여직원 중 최고 월급은 얼마인지 조사하라.  
> Q2. 총무부 직원이 최초로 입사한 날짜를 구하라.

In [9]:
select(cursor, text = 'A1.', table_name = 'tStaff',
       cond = 'gender = "여"', column = 'max(salary)')

select(cursor, text = 'A1. REF', table_name = 'tStaff', cond = 'gender = "여"', 
       order = 'salary desc', column = 'name, gender, salary')


select(cursor, text = 'A2.', table_name = 'tStaff',
       cond = 'depart = "총무부"', column = 'min(joindate)')

select(cursor, text = 'A2. REF', table_name = 'tStaff', cond = 'depart = "총무부"', 
       order        = 'joindate', column = 'name, depart, joindate')

[query] select max(salary) from tStaff where gender = "여";

[A1.]
(400,)


[query] select name, gender, salary from tStaff where gender = "여" order by salary desc;

[A1. REF]
('신사임당', '여', 400)
('유관순', '여', 380)
('논개', '여', 340)
('선덕여왕', '여', 315)
('허난설헌', '여', 285)
('황진이', '여', 275)


[query] select min(joindate) from tStaff where depart = "총무부";

[A2.]
(datetime.date(2000, 2, 3),)


[query] select name, depart, joindate from tStaff where depart = "총무부" order by joindate;

[A2. REF]
('김유신', '총무부', datetime.date(2000, 2, 3))
('이사부', '총무부', datetime.date(2000, 2, 3))
('정몽주', '총무부', datetime.date(2010, 9, 16))
('이율곡', '총무부', datetime.date(2016, 3, 8))
('정약용', '총무부', datetime.date(2020, 3, 14))
('대조영', '총무부', datetime.date(2020, 7, 7))




In [10]:
conn.close()

# **99. 참고자료**
## **99-1. 도서** 
- 소문난 명강의 - 김상형의 SQL 정복 | 김상형 저 / 한빛 미디어

## **99-2. 논문, 학술지**

## **99-3. 웹 사이트**

## **99-4. 데이터셋 출처**
